In [2]:
import gensim
import pandas as pd
import pickle
import glob
import numpy as np
import sys
sys.path.append('../src/')
from text_preprocessing import *
from utils import *

In [2]:
github_data = pd.concat([pd.read_csv(filename) for filename in glob.glob('../data/raw/github_data/*')])

/Users/kstathou/anaconda/envs/nesta/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/kstathou/anaconda/envs/nesta/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/kstathou/anaconda/envs/nesta/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
github_data.shape

(19159174, 17)

## Text preprocessing

In [4]:
github_data = github_data[github_data['project_description'].isnull()==False]
github_data.reset_index(inplace=True, drop=True)

In [5]:
nlproc = TextPreprocessing()

In [6]:
%%time
documents = [flatten_lists(nlproc.tokenize_document(doc)) for doc in list(github_data['project_description'])]

CPU times: user 24min 26s, sys: 26.2 s, total: 24min 52s
Wall time: 25min 12s


In [10]:
%%time
# Find bigrams
docs = nlproc.bigrams(documents)
project_ids = list(github_data['project_id'])

CPU times: user 8min 46s, sys: 1min 14s, total: 10min 1s
Wall time: 10min 26s


In [11]:
# Save docs 
with open('../data/interim/docs_gh.pickle', 'wb') as h:
    pickle.dump(docs, h)
    
# Save row ids
with open('../data/interim/project_ids_gh.pickle', 'wb') as h:
    pickle.dump(project_ids, h)

## Training word2vec and doc2vec

In [1]:
w2v = gensim.models.Word2Vec(docs, size=350, window=10, min_count=2, iter=20)

In [ ]:
w2v.save('../models/w2v_gh.word2vec')

In [4]:
w2v.most_similar(['machine_learning'], topn=10)

[('deep_learning', 0.66623854637146),
 ('classification', 0.6167000532150269),
 ('supervised_learning', 0.596238374710083),
 ('data_science', 0.5827585458755493),
 ('computer_vision', 0.5490185022354126),
 ('sklearn', 0.5454097390174866),
 ('neural_network', 0.5431226491928101),
 ('logistic_regression', 0.5427300930023193),
 ('mining', 0.5408807992935181),
 ('neural_networks', 0.5408588647842407)]

In [5]:
w2v.most_similar(['iot'], topn=10)

[('internet_things', 0.6005114912986755),
 ('azure_iot', 0.5117706060409546),
 ('electric_imp', 0.4698876142501831),
 ('smart_home', 0.4650612473487854),
 ('watson_iot', 0.44937294721603394),
 ('internet_connected', 0.44580090045928955),
 ('sensor', 0.44355714321136475),
 ('tessel', 0.44221973419189453),
 ('home_automation', 0.44139182567596436),
 ('particle_photon', 0.4413098692893982)]

In [ ]:
# create Tagged Documents (Gensim object) that will be used as input for the doc2vec
tagged_tags = nlproc.tagged_documents(documents=docs, labels=project_ids)

In [ ]:
%%time
d2v = gensim.models.doc2vec.Doc2Vec(documents=tagged_tags, size=350, min_count=2, 
                                    window=30, iter=30, dbow_words=1, dm=0)